# QAOA Weighted MAXSAT

Gahwon Lee, Andrew Tan

In [9]:
import qiskit
import numpy as np

## Introduction

### Weighted MAXSAT (3-SAT)

Given a boolean formula in conjunctive normal form as well as weights to each of those clauses, find the set of boolean variables that maximize the weights. For example, if a clause with weight 3 is true, then add 3 to the total weight. Each clause can contain exactly three boolean variables.

### QAOA

Quantum Approximation Optimization Algorithm is designed to approximate the Quantum Adiabatic Algorithm by splitting up different time slices into separate gates. Since it approximates the QAA, as the number of gates increases to infinity, the QAOA result will also increase to the ideal state.

## Methods

### Encoding Qubits

The bitstring corresponding to the output of QAOA will be the input qubits. Each qubit will either be 1 for true or 0 for false. 

### Cost and Driver Hamiltonians *C* and *B*

(3-SAT) $C = \sum_{m=1}^M I-\frac{1}{2}((I_1 \pm Z_1)\otimes (I_2 \pm Z_2)\otimes (I_3 \pm Z_3))$

(2-SAT) $C = \sum_{m=1}^M I-\frac{1}{2}((I_1 \pm Z_1)\otimes (I_2 \pm Z_2))$

$B = \sum_{k=1}^n X_k$

### Driver Gate W

$W(\beta) = \mul_{k=1}^n H_k

In [2]:
def ref_gate(n, qc, q, beta):
    # W gate
    for i in range(0,n):
        qc.h(q[i])
    for i in range(0,n):
        qc.cx(q[i],q[1])
        
    # apply phase gate
    qc.u1(2*beta, q[1])
    
    for i in range(n-1,-1,-1):
        qc.cx(q[i],q[1])
    for i in range(0,n):
        qc.h(q[i])

### Cost Gate V

$V(\gamma) = ...

In [ ]:
def constraint_gate(n, qc, q, gamma, x, y, clauses):
    # V gate
    # 2-SAT
    
    flip = 1 # adjust this flip variable
    qc.cx(q[x],q[y])
    qc.u1(flip*-1*gamma*weight,q[y])
    qc.cx(q[x],q[y])
    
    flip = 1 # adjust this flip variable
    qc.u1(flip*-1*gamma*weight,q[x])
    
    flip = 1 # adjust this flip variable
    qc.u1(flip*-1*gamma*weight,q[y])
    
    '''
    # 3-SAT
    for clause in clauses:
        # clause[0] signifies x ... clause[2]=z
        flip = 2 * ((clause[0] + clause[1] + clause[2]) % 2)-1 #if flip is true we do -1 otherwise 1
        
        qc.cx(q[1],q[0])
        qc.cx(q[2],q[0])
        qc.cx(q[3],q[0])
        qc.u1(flip * -1*2*gamma,q[0])
        qc.cx(q[3],q[0])
        qc.cx(q[2],q[0])
        qc.cx(q[1],q[0])
        
        flip = -2*((clause[1] + clause[2]) %2)+1
        
        qc.cx(q[2],q[0])
        qc.cx(q[3],q[0])
        qc.u1(flip * -1*2*gamma,q[0])
        qc.cx(q[3],q[0])
        qc.cx(q[2],q[0])
        
        flip = -2*((clause[0] + clause[2]) %2)+1
        
        qc.cx(q[1],q[0])
        qc.cx(q[3],q[0])
        qc.u1(flip * -1*2*gamma,q[0])
        qc.cx(q[3],q[0])
        qc.cx(q[1],q[0])
        
        flip = -2*((clause[0] + clause[1]) %2)+1
        
        qc.cx(q[1],q[0])
        qc.cx(q[2],q[0])
        qc.u1(flip * -1*2*gamma,q[0])
        qc.cx(q[2],q[0])
        qc.cx(q[1],q[0])
        
        qc.u1((clause[2] * -2 + 1) * gamma,q[3])
        qc.u1((clause[1] * -2 + 1) * gamma,q[2])
        qc.u1((clause[0] * -2 + 1) * gamma,q[1])
        
    '''

### Calculating the Expectation
Expectation = <gamma,beta|C|gamma,beta>

In [14]:
def expectation(gamma, beta, clauses):
    n = 3
    q = qiskit.QuantumRegister(n+1)
    c = qiskit.ClassicalRegister(n+1)
    qc = qiskit.QuantumCircuit(q,c)
    # q[0] is output qubit
    # q[1..n] are input qubits
    
    p = 1
    for pp in range(p):
        constraint_gate(n, qc, q, gamma, clauses)
        ref_gate(n, qc, q, beta)
    
    qc.measure(q, c)
    
    repetitions = 128
    job = qiskit.execute(qc, backend='local_qasm_simulator', shots=repetitions)
    result = job.result().get_counts(qc)
    
    answer = 0
    for key in result.keys():
        if key[0] == '1':
            answer += result[key] / repetitions
    
    return answer

### Classically Optimizing *γ* and *β*

In [16]:
def targetFunc(params):
    gamma, beta = params
    clauses = [[0,0,0],[0,0,1],[0,1,0]]
    return -qaoa(gamma, beta, clauses)

from scipy.optimize import minimize

x0 = np.array([1.3, 0.7])
res = minimize(targetFunc, x0, method='nelder-mead', options={'xtol': 1e-8, 'disp': True})

## Results and Discussion

### Generate the following figures, and interpret them.  Make sure to include error bars.

blah

### For several random problem instances, plot the cost of the output state.

#### Plot average, maximum and minimum cost.  How do these compare?

blah

### Fix all parameters except one, and plot cost as a function of the free parameter.

#### Will a classical optimizer always find the best solution?

blah

## Conclusion

### Was QAOA effective in finding decent solutions?

no

### What % of the optimum solution did QAOA achieve?

0%

### Does the technique have any issues?

maybe